<a href="https://colab.research.google.com/github/vivektheeyarath/my-make-more/blob/dev/build_makemore_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

!wget https://raw.githubusercontent.com/karpathy/makemore/morelm/names.txt

In [2]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
words[:8]

In [ ]:
len(words)

In [ ]:
# build vocabulary of chars and mappings to/from integers
chars = sorted(set(list(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
#stoi.items()
itos = {i:s for s,i in stoi.items()}#
print(itos)

In [ ]:
# build the dataset
block_size = 3
X, Y = [], []
for w in words[:5]:
  print(w)
  context = [0]*block_size
  for ch in w+'.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] +[ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [ ]:
X.shape, X.dtype, Y.shape, Y.dtype

In [8]:
C = torch.randn(27,2)

In [ ]:
emb = C[X]
emb.shape

In [10]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [ ]:
h = torch.tanh(emb.view(32,6))@W1 + b1
h

In [ ]:
h.shape

In [13]:
W2 = torch.randn(100,27)
b2 = torch.randn(27)

In [14]:
logits = h @ W2 + b2

In [ ]:
logits.shape

In [16]:
counts = logits.exp()

In [17]:
prob = counts / counts.sum(1, keepdim=True)

In [ ]:
prob.shape

In [ ]:
prob

In [ ]:
prob[0].sum()

In [ ]:
prob[torch.arange(32), Y]

In [ ]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

In [ ]:
# Now -- making the code so far respectable --
X.shape, Y.shape #dataset

In [24]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6,100), generator=g) #
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100,27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [ ]:
sum(p.nelement() for p in parameters)

In [ ]:
# Forward pass
emb = C[X] #(32, 3, 2)
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
#counts = logits.exp()
#prob = counts / counts.sum(1, keepdims=True)
#loss = -prob[torch.arange(32), Y].log().mean()
F.cross_entropy(logits, Y)